# Interactive Visualizer (InViz)
This is the notebook where I'm currently developing the visualizer. I'm using Holoviews and Panel to construct the dashboard and implement the interactivity.

In [ ]:
import inviz as nv

In [ ]:
import holoviews as hv
from holoviews import dim, opts, streams
from holoviews.selection import link_selections
# import hvplot.pandas
import pandas as pd
from itertools import combinations
import numpy as np
from tqdm import trange, tqdm
import re
import panel as pn
import spatialpandas
import os
from bokeh.models import HoverTool
from classy import Class
import matplotlib.pyplot as plt
import copy
from multiprocessing import Pool
from typing import Callable

# pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="stretch_width")

# hv.Store.set_current_backend('bokeh')
# pn.extension('tabulator')
hv.extension('bokeh')
pn.extension()


## Function & Class Definitions

In [ ]:
# read in the .paramnames file and put it into list format
def load_params(filename):
    params_list = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            params = re.split(' \t ', line)
            params_list.append(params)
    return [item[0] for item in params_list], [item[1] for item in params_list]


# create a DataFrame with the chain files as rows and use a list of parameters as the column names
def load_data(filename, column_names):
    data = np.loadtxt(filename)
    df = pd.DataFrame(data[:,1:], columns=column_names)
    return df


def run_class(selection):
    # run class on the user's selection
    cosmo = Class()
    cosmo.set(selection)
    cosmo.set({'output':'mPk, tCl, pCl, lCl','P_k_max_1/Mpc':3.0, 'lensing':'yes'})
    cosmo.compute()

    # set variables for matter power spectrum and lensed CMB angular power spectra
    kk = np.logspace(-4,np.log10(3),1000)
    Pk = []
    h = cosmo.h()
    for k in kk:
        Pk.append(cosmo.pk(k*h,0.)*h**3)
    Pk = np.array(Pk)
    l = np.array(range(2,2501))
    factor = l*(l+1)/(2*np.pi)
    lensed_cl = cosmo.lensed_cl(2500)
    
    results = {'k': kk, 'Pk': Pk, 'l': l, 'Cl_tt': factor*lensed_cl['tt'][2:], 'Cl_ee': factor*lensed_cl['ee'][2:]}
    
    cosmo.struct_cleanup()
    cosmo.empty()
    return results


def compute_residuals(index, sample, sample_CDM):
    selection = sample.iloc[[index]].to_dict('index')
    selection_CDM = sample_CDM.iloc[[index]].to_dict('index')
    if __name__ == '__main__':
        with Pool() as p:
            [mycosmo, LambdaCDM] = p.map(run_class, [selection[index], selection_CDM[index]])
    else:
        mycosmo = run_class(selection[index])
        LambdaCDM = run_class(selection_CDM[index])

    pk_residuals = (mycosmo['Pk'] - LambdaCDM['Pk'])/LambdaCDM['Pk']*100
    cl_tt_residuals = (mycosmo['Cl_tt'] - LambdaCDM['Cl_tt'])/LambdaCDM['Cl_tt']*100
    cl_ee_residuals = (mycosmo['Cl_ee'] - LambdaCDM['Cl_ee'])/LambdaCDM['Cl_ee']*100
    
    residuals = [
        {'k': mycosmo['k'], 'pk_residuals': pk_residuals}, 
        {'l': mycosmo['l'], 'cl_tt_residuals': cl_tt_residuals}, 
        {'l': mycosmo['l'], 'cl_ee_residuals': cl_ee_residuals},
    ]
    return residuals

In [ ]:
#  given a param name, find corresponding latex-formatted param name
def lookup_latex_label(param, latex_dict):
    try:
        latex_param = latex_dict[param]
        label = r'$${}$$'.format(latex_param)
        return label
    except KeyError:
        label = param
        return label


def viz(
    data, 
    observables: list = None, 
    show_observables: bool = True, 
    latex_dict: dict = None
):
    # handle default case of no latex paramname dictionary
    if latex_dict is None:
        latex_dict = dict()
    # setting Panel widgets for user interaction
    variables = data.columns.values.tolist()
    var1 = pn.widgets.Select(value=variables[1], name='Horizontal Axis', options=variables)
    var2 = pn.widgets.Select(value=variables[2], name='Vertical Axis', options=variables)
    cmap_var = pn.widgets.Select(value=variables[0], name='Colormapped Parameter', options=variables)
    cmap_option = pn.widgets.Checkbox(value=True, name='Show Colormap', align='end')

    # function for generating the scatter plot, given 2 dimensions as x and y axes, and an additional dimension to colormap
    # to the points on the plot. Also has an option to show or hide the colormap
    def plot_data(kdim1, kdim2, colordim, showcmap):
        if showcmap == True:
            cmapping = opts.Points(color=dim(colordim),
                colorbar=True,
                cmap='GnBu_r')
        else:
            cmapping = opts.Points(color='grey', colorbar=True)
        hover = HoverTool(tooltips=None)
        xlabel = lookup_latex_label(kdim1, latex_dict)
        ylabel = lookup_latex_label(kdim2, latex_dict)
        popts = opts.Points(
            bgcolor='#E5E9F0',
            fontscale=1.1,
            xlabel=xlabel,
            ylabel=ylabel,
            toolbar='above',
            line_color='black',
            #alpha=0.75, selection_alpha=1, nonselection_alpha=0.1,
            tools=[hover, 'box_select','lasso_select','tap'],
            size=7)
        points = hv.Points(data, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return points
    
    # bind the widget values to the plotting function so it gets called every time the user interacts with the widget
    # call the bound plotting function inside a holoview DynamicMap object for interaction
    interactive_points = pn.bind(plot_data, kdim1=var1, kdim2=var2, colordim=cmap_var, showcmap=cmap_option)
    points_dmap = hv.DynamicMap(interactive_points, kdims=[]).opts(width=500, height=400, framewise=True)
    
    # define a stream to get a list of all the points the user has selected on the plot
    selection = streams.Selection1D(source=points_dmap)
    
    # formatting the table using plot hooks
    def hook(plot, element):
        plot.handles['table'].autosize_mode = "none"
        for column in plot.handles['table'].columns:
            column.width = 100
    
    # function to generate a table of all the selected points
    def make_table(kdim1, kdim2, colordim):
        table_options = opts.Table(height=300, width=1000, hooks=[hook], bgcolor='#f5f5f5')
        table = hv.DynamicMap(lambda index: hv.Table(data.iloc[index], kdims=[kdim1, kdim2, colordim]), streams=[selection])
        return table.opts(table_options).relabel('Selected Points')
    
    
    # generate the table
    selected_table = pn.bind(make_table, kdim1=var1, kdim2=var2, colordim=cmap_var)
    
    #table_stream = streams.Selection1D(source=selected_table)
    
    # handles the null selection case and multiple selections
    plots = {}
    # get total number of plots to draw from list of observables
    plotting_info = {}
    for each in observables:
        for i in range(len(each.name)):
            if each.plot_opts is None:
                specific_opts = None
            elif each.plot_opts[i] is not None:
                specific_opts = each.plot_opts[i]
            plotting_info[each.name[i]] = {'type': each.plot_type[i], 'opts': specific_opts}
    def plot_observables(index):
        if not index:
            plots_list = []
            for name in plotting_info:
                hv_type = getattr(hv, plotting_info[name]['type'])
                empty_plot = hv_type(np.random.rand(0, 2)).relabel(f'{name} - No Selection').opts(height=400, width=500, fontscale=1.1, framewise=True)
                if plotting_info[name]['opts'] is not None:
                    empty_plot.opts(plotting_info[name]['opts'])
                plots_list.append([empty_plot])
        else:
            new_index = [x for x in index if x not in list(plots.keys())]
            for element in new_index:
                new_plots = []
                for each in observables:
                    new_plots.extend(each.generate_plot(element))
                plots[element] = {plot.label: plot for plot in new_plots}
            
            plots_list = []
            for index_item in index:
                plot_types = list(plots[index_item].keys())

            for plot_item in plot_types:
                same_type = [plots[key][plot_item] for key in index]
                plots_list.append(same_type)
            
        layout = hv.Layout()
        for list_of_plots in plots_list:
            overlay = hv.Overlay(list_of_plots).opts(show_legend=False)
            layout = layout + overlay
        layout.opts(shared_axes=False).cols(3)
        return layout
    
    
    # put it all together using Panel
    dashboard = pn.Column(pn.Row(var1, var2, cmap_var, cmap_option), pn.Row(points_dmap, selected_table))
    
    if show_observables == True:
        observables_dmap = hv.DynamicMap(plot_observables, streams=[selection]).opts(framewise=True)
        observables_pane = pn.panel(observables_dmap)
        dashboard = pn.Column(dashboard, observables_pane)
    
    return dashboard

In [ ]:
class Observable:
    """
    Observable class for InViz.
    
    Parameters
    ----------
    name: string or list of strings
        specifies the display name of the observable for things like plot titles
    parameters: dict-like or list of dict-likes
        the data to associated with that observable. can be python dict (or pandas DataFrame)
        whose keys (or column names) will be used for things like plot axis labels. 
    latex_labels: dict or list of dicts
        key: value -> parameter label: latex version. parameter label must match the
        corresponding one in the parameters dict
    myfunc: callable
        a user-provided function that returns parameters. can return more than one
        set of parameters if the "grouped" option is True
    myfunc_args: tuple
        arguments for user-provided function
    grouped: boolean
        specifies if user-provided function returns more than one set of parameters
    plot_type: string
        specifies how the data should be visualized. currently can pick either 'Curve'
        or 'Scatter'
    plot_opts: holoviews Options object
        customization options for the observable plot. see Holoviews documentation
    """
    
    def __init__(
        self, 
        name: str | list[str] = None, 
        parameters: dict | list[dict] = None, 
        myfunc: Callable = None,
        myfunc_args: tuple = None, 
        grouped: bool = False, 
        plot_type: str | list[str] = None,
        plot_opts: type[opts] | list[type[opts]] = None,
        latex_labels: dict = None
    ):
        self.name = [name]
        self.parameters = parameters
        if parameters is not None:
            self.parameters = [parameters]
        self.myfunc = myfunc
        self.myfunc_args = myfunc_args
        self.plot_type = plot_type
        if plot_type is not None:
            self.plot_type = [plot_type]
        self.plot_opts = plot_opts
        if plot_opts is not None:
            self.plot_opts = [plot_opts]
        self.grouped = grouped
        self.latex_labels = latex_labels
        if self.grouped:
            self.name = name
            self.parameters = parameters
            self.plot_type = plot_type
            self.plot_opts = plot_opts
        self.number = len(self.name)
    
    def properties(self):
        if self.grouped:
            print("InViz Grouped Observable")
            for i in range(len(self.name)):
                print(f"\t- Observable {i+1}: {self.name[i]}")
        else:
            print("InViz Observable")
            print(f"Name: {self.name}")
        
    def generate_plot(self, index: int):
        self.plots_list = []
        if self.myfunc and self.myfunc_args is not None:
            computed_data = self.myfunc(index, *self.myfunc_args)
            self.number = len(computed_data)
        for i in range(0, self.number):
            hv_element = getattr(hv, self.plot_type[i])
            if self.parameters is not None:
                dataset = self.parameters[i]
                kdim = list(dataset.keys())[0]
                vdim = list(dataset.keys())[1]
                plot = hv_element(dataset[index], kdim, vdim, label=self.name[i])
            elif computed_data:
                dataset = computed_data[i]
                kdim = list(dataset.keys())[0]
                vdim = list(dataset.keys())[1]
                plot = hv_element(dataset, kdim, vdim, label=self.name[i])
            # set defaults
            plot.opts(
                height=400,
                width=500,
                fontscale=1.1,
                xlabel=lookup_latex_label(kdim, self.latex_labels), 
                ylabel=lookup_latex_label(vdim, self.latex_labels),
                framewise=True
            )
            # add user defined customizations
            if self.plot_opts and self.plot_opts[i] is not None:
                plot.opts(self.plot_opts[i])
            self.plots_list.append(plot)
        return self.plots_list
        
    def draw_plot(self, index, observable_name=None):
        layout = hv.Layout(self.generate_plot(index))
        return layout.opts(shared_axes=False)
        
    

## Visualizing
Now we just need to load in our data, convert it into a format that CLASS accepts, and call `viz` to produce the dashboard.

In [ ]:
# Read in data
loglkl = ['loglkl']
param_names, latex_params = load_params('../data/chains_planckbossdes_1MeV/2022-11-16_3200000_.paramnames')
params_latex_form = dict(zip(param_names, latex_params))
column_names = loglkl + param_names
df = pd.DataFrame(columns=column_names)
for i in trange(1,5):
    temp = load_data('../data/chains_planckbossdes_1MeV/2022-11-16_3200000__{}.txt'.format(i), column_names=column_names)
    df = pd.concat([df,temp]).reset_index(drop=True)

In [ ]:
# prepare data for CLASS computation
# remove nuisance parameters
classy_input = df.drop(columns=['loglkl', 'z_reio', 'A_s', 'sigma8', '100theta_s', 'A_cib_217', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 'galf_TE_A_217', 'calib_100T', 'calib_217T', 'A_planck', 'b^{(1)}_1', 'b^{(1)}_2', 'b^{(1)}_{G_2}', 'b^{(2)}_1', 'b^{(2)}_2', 'b^{(2)}_{G_2}', 'b^{(3)}_1', 'b^{(3)}_2', 'b^{(3)}_{G_2}', 'b^{(4)}_1', 'b^{(4)}_2', 'b^{(4)}_{G_2}'])
classy_input['omega_b'] = df['omega_b'] * 1e-2
classy_input['sigma_dmeff'] = df['sigma_dmeff'] * 1e-25
classy_input = classy_input.rename(columns={'H0':'h'})
classy_input['h'] = classy_input['h'] * 1e-2
classy_input['f_dmeff'] = 0.1
classy_input['npow_dmeff'] = 0.0
classy_input['Vrel_dmeff'] = 0.0
classy_input['dmeff_target'] = 'baryons'
classy_input['m_dmeff'] = 1e-3

# format for CDM version
#classy_CDM = classy_input.drop(columns=['sigma_dmeff', 'omega_cdm', 'npow_dmeff', 'Vrel_dmeff', 'dmeff_target', 'm_dmeff'])
# classy_CDM = classy_CDM.rename(columns={'omega_dmeff':'omega_cdm'})
classy_CDM = classy_input.drop(columns=['sigma_dmeff', 'npow_dmeff', 'Vrel_dmeff', 'dmeff_target', 'm_dmeff'])


# slice for fast computation
classy_input_slice = classy_input[::500].reset_index(drop=True)
classy_CDM_slice = classy_CDM[::500].reset_index(drop=True)
df_slice = df[::500].reset_index(drop=True)

In [ ]:
interp_df = pd.read_pickle('../data/trey_uvlf/bouwens_2023_data.pkl')
binned_df = pd.read_pickle('../data/trey_uvlf/bouwens_2023_data_binned.pkl')

In [ ]:
params_df = binned_df[['alphaOutflow', 'alphaStar', 'like', 'timescale', 'velocityOutflow']]
lumfunc_df = binned_df[['uvlf_Muv', 'uvlf_z10.5', 'uvlf_z12.6', 'uvlf_z8.7']]
uvlf = {}
for row in lumfunc_df.index:
    uvlf[row] = {
        'z = 10.5': {'uvlf_Muv': lumfunc_df['uvlf_Muv'][row], 'uvlf_z10.5': lumfunc_df['uvlf_z10.5'][row]}, 
        'z = 12.6': {'uvlf_Muv': lumfunc_df['uvlf_Muv'][row], 'uvlf_z12.6': lumfunc_df['uvlf_z12.6'][row]}, 
        'z = 8.7': {'uvlf_Muv': lumfunc_df['uvlf_Muv'][row], 'uvlf_z8.7': lumfunc_df['uvlf_z8.7'][row]}, 
    }
reshaped_uvlf = pd.DataFrame.from_dict(uvlf, orient='index')

### The Viz Function

The viz function has three arguments:
- __data__: (Pandas DataFrame) the data you want shown as a scatter plot
- __myfunction__: (function) your function that computes some observables based on a data point
- __function_args__: (tuple) the arguments of that function
- __show_observables__: (Boolean) whether you want to see the observable plots or not 

Be aware that if you pass data with categorical values (i.e. strings) in the first argument, the visualizer will break. It's fine to do so for the CLASS arguments though.

In [ ]:
cosmo_copts = opts.Curve(width=500, height=400, logx=True, padding=0.1, fontscale=1.1, color=hv.Cycle('GnBu'), bgcolor='#22262F', framewise=True)
cosmo_latex = {
    'k': 'k~[h/\mathrm{Mpc}]',
    'pk_residuals': '(P(k)-P_{CDM}(k))/P_{CDM}(k)*100~[\%]',
    'l': '\ell',
    'cl_tt_residuals': '(C_{\ell}^{TT}-C_{\ell, CDM}^{TT})/C_{\ell, CDM}^{TT}*100~[\%]',
    'cl_ee_residuals': '(C_{\ell}^{EE}-C_{\ell, CDM}^{EE})/C_{\ell, CDM}^{EE}*100~[\%]',
}
cosmo_observables = Observable(
    name=[
        'P(k) Residuals', 
        'Cl_TT Residuals', 
        'Cl_EE Residuals', 
    ], 
    myfunc=compute_residuals,
    myfunc_args=(classy_input_slice, classy_CDM_slice), 
    plot_type=[
        'Curve', 
        'Curve', 
        'Curve', 
    ],
    plot_opts=[
        cosmo_copts, 
        cosmo_copts, 
        cosmo_copts, 
    ],
    latex_labels=cosmo_latex,
    grouped=True
)

In [ ]:
viz(data=df_slice, observables=[cosmo_observables], latex_dict=params_latex_form)

In [ ]:
uvlf_scatter_opts = opts.Scatter(ylim=(1e-11, None), logy=True, invert_xaxis=True, height=400, width=500, size=5, marker='square')
uvlf_curve_opts = opts.Curve(ylim=(1e-11, None), logy=True, invert_xaxis=True, height=400, width=500)
uvlf_observables = Observable(
    name=[
        'z = 10.5', 
        'z = 12.6', 
        'z = 8.7'
    ], 
    parameters=[
        (lumfunc_df['uvlf_Muv'], lumfunc_df['uvlf_z10.5']), 
        (lumfunc_df['uvlf_Muv'], lumfunc_df['uvlf_z12.6']), 
        (lumfunc_df['uvlf_Muv'], lumfunc_df['uvlf_z8.7'])
    ],
    parameter_labels=[
        (r'$$\text{UV Magnitude}$$', r'$$\text{Luminosity Function}$$'), 
        (r'$$\text{UV Magnitude}$$', r'$$\text{Luminosity Function}$$'), 
        (r'$$\text{UV Magnitude}$$', r'$$\text{Luminosity Function}$$')
    ], 
    plot_type=[
        'Curve', 
        'Curve', 
        'Curve', 
    ],
    plot_opts=[
        uvlf_curve_opts, 
        uvlf_curve_opts, 
        uvlf_curve_opts, 
    ],
    grouped=True
)

In [ ]:
z10_5 = Observable(
    name='z = 10.5', 
    parameters=(lumfunc_df['uvlf_Muv'], lumfunc_df['uvlf_z10.5']), 
    parameter_labels=(r'$$\text{UV Magnitude}$$', r'$$\text{Luminosity Function}$$'),
    plot_type='Curve',
    plot_opts=uvlf_curve_opts
)

In [ ]:
viz(data=params_df, observables=[z10_5])

## WIP
The below cells are being used to further develop the visualizer.

In [ ]:
test_params = {'omega_b': 2.268118e-2, 
               'omega_dmeff': 0.1221163, 
               'ln10^{10}A_s': 3.048006, 
               'n_s': 0.9670508, 
               'tau_reio': 0.05659527, 
               'sigma_dmeff': 1.193e-25, 
               'Omega_Lambda': 0.6772064, 
               'YHe': 0.2479716, 
               'h': 67.13298e-2,  
               'omega_cdm': 1e-15, 
               'npow_dmeff': 0, 
               'Vrel_dmeff': 0, 
               'dmeff_target': 'baryons', 
               'm_dmeff': 0.001}
test_params_CDM = {'omega_b': 2.268118e-2, 
                   'omega_cdm': 0.1221163, 
                   'ln10^{10}A_s': 3.048006, 
                   'n_s': 0.9670508, 
                   'tau_reio': 0.05659527, 
                   'Omega_Lambda': 0.6772064, 
                   'YHe': 0.2479716, 
                   'h': 67.13298e-2}

In [ ]:
args = (classy_input_slice, classy_CDM_slice)
resids19 = compute_residuals(19, *args)
resids20 = compute_residuals(20, *args)
resids21 = compute_residuals(21, *args)

resids = {19: resids19, 20: resids20, 21: resids21}

In [ ]:
resids_df = pd.DataFrame.from_dict(resids, orient='index')
resids_df

In [ ]:
reshaped_uvlf

In [ ]:
data = df_slice
popts = opts.Points(
    toolbar='above',
    line_color='black',
    #alpha=0.5, selection_alpha=1, nonselection_alpha=0.1,
    tools=['box_select','lasso_select','tap'],
    size=7)
numerical_points = hv.Points(data, kdims=['omega_b', 'omega_cdm']).opts(popts)
selection = streams.Selection1D(source=numerical_points)

selections = []
def record_selections(index):
    new_index = [x for x in index if x not in list(set(selections))]
    prev_index = [x for x in index if x not in new_index]
    new = hv.Points(df_slice.iloc[new_index], kdims=['omega_b', 'omega_cdm']).relabel('new')
    old = hv.Points(df_slice.iloc[prev_index], kdims=['omega_b', 'omega_cdm']).relabel('old')
    if None not in index:
        selections.extend(index)
    return new * old

def show_prev_selections(index):
    if not index:
        return hv.Points(df_slice.iloc[index], kdims=['omega_b', 'omega_cdm']).relabel('no selection')

    new_index = [x for x in index if x not in list(set(selections))]
    prev_index = [x for x in index if x not in new_index]
    # old_index = prev_index + [x for x in list(set(selections)) if x not in prev_index]
    return hv.Points(df_slice.iloc[prev_index], kdims=['omega_b', 'omega_cdm']).relabel('previous')

def show_selection_record(index):  
    new_index = [x for x in index if x not in list(set(selections))]
    prev_index = [x for x in index if x not in new_index]
    old_index = prev_index + [x for x in list(set(selections)) if x not in prev_index]
    return hv.Points(df_slice.iloc[old_index], kdims=['omega_b', 'omega_cdm']).relabel('old')


responsive_dmap = hv.DynamicMap(record_selections, streams=[selection]).opts(popts)
prev_dmap = hv.DynamicMap(show_prev_selections, streams=[selection]).opts(popts)
old_dmap = hv.DynamicMap(show_selection_record, streams=[selection]).opts(popts)

numerical_points + responsive_dmap + old_dmap

In [ ]:
index = selection.contents['index']
new_index = [x for x in index if x in list(set(selections))]
prev_index = [x for x in index if x not in new_index]
old_index = prev_index + [x for x in list(set(selections)) if x not in prev_index]
print(f'new index: \n{new_index} \nprevious index: \n{prev_index} \nold index: \n{old_index}')

In [ ]:
from holoviews.operation import decimate
decimate.max_samples=2000
points = hv.Points(df, kdims=['omega_b', 'omega_cdm']).opts(width=500, height=400)
decimate(points)

In [ ]:
decimate(points) + hv.Points(df_slice, kdims=['omega_b', 'omega_cdm']).opts(width=500, height=400)

In [ ]:
def format_list(l):
    print(' '.join(sorted([k for k in l if not k.endswith('_r')])))
format_list(hv.Cycle.default_cycles.keys())
format_list(hv.Palette.colormaps.keys())

In [ ]:
def plot_scatter_table(data, params, plots):
    # kwargs:
    # data: a pandas DataFrame
    # params: list of parameters that correspond to column names in the DataFrame
    # plots: the number of plots to display
    
    # generate a list of all pairs of the parameters
    pairs = [list(comb) for comb in combinations(params, 2)]
    
    # create linked selections
    ls = link_selections.instance()
    
    layout = hv.Layout()
    
    for param_a, param_b in pairs[:plots]:
        # vdims = [e for e in params if e not in (param_a, param_b)]
        # ^^^ uncomment the above and add ", vdims" to the argument of hv.Dataset if desired
        ds = hv.Dataset(data, [param_a, param_b])
        pts = hv.Points(ds).opts(
            opts.Points(color='black', size=2))
        bivar = hv.Bivariate(data[[param_a,param_b]].values, [param_a,param_b], []).opts(
            opts.Bivariate(bandwidth=0.5,
                           cut=0,cmap="blues",
                           levels=5,
                           colorbar=False,
                           show_legend=False,
                           filled=True,
                           toolbar='above',
                           width=350,
                           alpha=0.75))
        layout += (ls(pts)*bivar).opts(width=300, height=300)
    
    layout = layout.cols(4)
    
    table = pn.widgets.Tabulator(data[params], disabled=True)
    selection = pn.widgets.Tabulator(disabled=True)
    try:
        table.selection = ls.selection_expr
    except ValueError:
        table.selection = []
    
    button_show_selection = pn.widgets.Button(name='Show selected rows only', button_type='primary', width_policy='auto')
    button_download_selection = pn.widgets.Button(name='Download selections as CSV', button_type='primary', width_policy='auto')
    button_reset = pn.widgets.Button(name='Reset', width_policy='auto')
    
    def show_selection(event):
        selection.value = table.selected_dataframe.sort_index()
    
    def download_selection(event):
        os.makedirs('data/selections', exist_ok=True)
        selection_df_sorted = table.selected_dataframe.sort_index()
        selection_df_sorted.to_csv('data/selections/selected_output.csv', index=False)

    def reset(event):
        table.selection = []
        selection.value = table.selected_dataframe

    button_show_selection.on_click(show_selection)
    button_download_selection.on_click(download_selection)
    button_reset.on_click(reset)
    dashboard = pn.Row(table, pn.Column(pn.Row(button_show_selection, button_download_selection, button_reset), selection))
    params = pn.Param(ls, parameters=['selection_mode'])
    scatter_table = pn.Column(params, pn.Row(layout), dashboard)
    return scatter_table

In [ ]:
params = ['omega_b', 'omega_dmeff', 'n_s', 'tau_reio', 'sigma_dmeff', 'H0', 'A_s', 'sigma8']
viz = plot_scatter_table(new_df, params, plots=4)
viz

In [ ]:
menu_items = list(ds.data)
menu_button = pn.widgets.MenuButton(name='Parameter', items=menu_items, button_type='primary')
text = pn.widgets.TextInput(value='Ready')

def b(event):
    text.value = f'Clicked menu item: "{event.new}"'
    
menu_button.on_click(b)

pn.Row(menu_button, text, height=300)

In [ ]:
ds = hv.Dataset(new_df, ['omega_b', 'sigma_dmeff'])
vdims = ['omega_b', 'omega_dmeff', 'ln10^{10}A_s', 'n_s', 'tau_reio', 'sigma_dmeff', 'Omega_Lambda', 'H0']
layout = hv.Layout()
for vdim in vdims:
    if vdim != ds.kdims[0].name and vdim != ds.kdims[1].name:
        pts = hv.Points(ds).opts(color=dim(vdim),
            colorbar=True,
            cmap='Magma',
            alpha=0.5, selection_alpha=1, nonselection_alpha=0.1,
            tools=['box_select','lasso_select','tap'],
            size=5,
            width=350,
            height=250).relabel(vdim)
        layout += pts

layout.opts(shared_axes=False).cols(3)

In [ ]:
def plot_data(kdim1, kdim2, colordim, showcmap):  
    popts = opts.Points(
        toolbar='above',
        line_color='black',
        #alpha=0.5, selection_alpha=1, nonselection_alpha=0.1,
        tools=['box_select','lasso_select','tap'],
        size=7)
    
    if classy_input_slice[colordim].dtype == object:
        if showcmap == False:
            cmapping = opts.Points(color='grey')
        cmapping = opts.Points(color=dim(colordim), cmap='Category20')
        categorical_points = hv.Points(classy_input_slice, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return categorical_points
    else:
        if showcmap == False:
            cmapping = opts.Points(color='grey')
        cmapping = opts.Points(color=dim(colordim), cmap='Viridis', colorbar=True)
        numerical_points = hv.Points(classy_input_slice, kdims=[kdim1, kdim2]).opts(popts, cmapping)
        return numerical_points
        

def plot_dist(kdim1):
    dist = hv.Distribution(new_df, kdim1).opts(
        alpha=1,
        color='blue')
    return dist

def plot_bivar(kdim1, kdim2):
    bivar = hv.Bivariate(classy_input_slice, kdims=[kdim1, kdim2]).opts(
        cmap='blues',
        bandwidth=1,
        cut=0,
        levels=5,
        filled=True)
    return bivar

def choose_dist(kdim1, kdim2):
    if kdim1 == kdim2:
        return hv.DynamicMap(plot_dist, kdim1=kdim1).opts(width=500, height=400, framewise=True)
    else:
        return hv.DynamicMap(plot_bivar, kdim1=kdim1, kdim2=kdim2).opts(width=500, height=400, framewise=True)

In [ ]:
variables = classy_input_slice.columns.values.tolist()
var1 = pn.widgets.Select(value=variables[0], name='Horizontal Axis', options=variables)
var2 = pn.widgets.Select(value=variables[1], name='Vertical Axis', options=variables)
cmap_var = pn.widgets.Select(value='omega_b', name='Colormapped Parameter', options=variables)
cmap_option = pn.widgets.Checkbox(value=False, name='Show Colormap', align='end')

In [ ]:
interactive_points = pn.bind(plot_data, kdim1=var1, kdim2=var2, colordim=cmap_var, showcmap=cmap_option)
interactive_dist = pn.bind(choose_dist, kdim1=var1, kdim2=var2)
dmap_points = hv.DynamicMap(interactive_points).opts(width=500, height=400, framewise=True)
#dmap_dist = hv.DynamicMap(interactive_dist).opts(width=500, height=400, framewise=True)
app = pn.Column(pn.Row(var1, var2, cmap_var, cmap_option, width = 700), dmap_points)
app

In [ ]:
choose_dist('omega_b','omega_cdm')

In [ ]:
table = hv.Table(new_df, kdims=['omega_b', 'omega_dmeff', 'sigma_dmeff'])
table_sel = streams.Selection1D(source=table)

In [ ]:
table_sel.contents

In [ ]:
# Set up the parameters of the problem.
ndim, nsamples = 3, 1000

# Generate some fake data.
np.random.seed(42)
data1 = np.random.randn(ndim * 4 * nsamples // 5).reshape(
    [4 * nsamples // 5, ndim]
)
data2 = 4 * np.random.rand(ndim)[None, :] + np.random.randn(
    ndim * nsamples // 5
).reshape([nsamples // 5, ndim])
data = np.vstack([data1, data2])

In [ ]:
param_names = ['frequency', 'phase', 'amplitude']
latex = ['\omega / 2\pi', '\phi', '\mathrm{amplitude}']
wavedf = pd.DataFrame(data, columns=param_names)
latex_dict = dict(zip(param_names, latex))

In [ ]:
from scipy import signal

def compute_waveforms(index, input_data):
    selection = input_data.iloc[[index]]
    x = np.linspace(-4*np.pi, 4*np.pi, 1000)
    angular_freq = 2*np.pi*selection['frequency'].iloc[0]
    phase = selection['phase'].iloc[0]
    amp = selection['amplitude'].iloc[0]
    sin = amp * np.sin(angular_freq*x + phase)
    sinc = amp * np.sinc(angular_freq*x/np.pi + phase)
    sawtooth = amp * signal.sawtooth(angular_freq * x + phase)
    waves = [
        {'x': x, 'sin(x)': sin}, 
        {'x': x, 'sinc(x)': sinc}, 
        {'x': x, 'sawtooth': sawtooth}
    ]
    return waves

In [ ]:
opts1 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), color=hv.Cycle('YlOrRd'), bgcolor='#151515')
opts2 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), color=hv.Cycle('PuBuGn'), bgcolor='#151515')
opts3 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), color=hv.Cycle('RdPu'), bgcolor='#f5f5f5')
waves_latex = {
    'x': 'x', 
    'sin(x)': '\sin{x}',
    'sinc(x)': '1/\sin{x}',
    'sawtooth': '\mathrm{Sawtooth~Wave}',
}

In [ ]:
waveforms = Observable(
    name=[
        'Sine',
        'Sinc',
        'Sawtooth'
    ],
    myfunc=compute_waveforms,
    myfunc_args=(wavedf,),
    grouped=True,
    plot_type=[
        'Curve',
        'Curve',
        'Curve',
    ],
    plot_opts=[
        opts1,
        opts2,
        opts3
    ],
    latex_labels=waves_latex
)

In [ ]:
def cosine(index, input_data):
    selection = input_data.iloc[[index]]
    x = np.linspace(-4*np.pi, 4*np.pi, 1000)
    angular_freq = 2*np.pi*selection['frequency'].iloc[0]
    phase = selection['phase'].iloc[0]
    amp = selection['amplitude'].iloc[0]
    cos = amp * np.cos(angular_freq*x + phase)
    waves = [
        {'x': x, 'cos(x)': cos}
    ]
    return waves

cosine_latex = {'x': 'x', 'cos(x)': '\cos{x}'}

coswav = Observable(
    name='Cosine',
    myfunc=cosine,
    myfunc_args=(wavedf,),
    plot_type='Curve',
    plot_opts=opts3,
    latex_labels=cosine_latex
)

In [ ]:
waveforms.parameters

In [ ]:
coswav.parameters

In [ ]:
viz(wavedf, [waveforms, coswav], latex_dict=latex_dict)

In [ ]:
cut = wavedf.drop(columns='frequency')

In [ ]:
pts = hv.Points(cut).opts(tools=['tap', 'box_select'], width=500, height=400, size=7, line_color='black')
selection = streams.Selection1D(source=pts)
pts

In [ ]:
observables_dmap = hv.DynamicMap(check_plot_observables, streams=[selection]).opts(framewise=True)

In [ ]:
observables_dmap

In [ ]:
selection.contents

In [ ]:
plots = {}
observables = [waveforms, coswav]
plotting_info = {}
for each in observables:
    for i in range(len(each.name)):
        if each.plot_opts is None:
            specific_opts = None
        elif each.plot_opts[i] is not None:
            specific_opts = each.plot_opts[i]
        plotting_info[each.name[i]] = {'type': each.plot_type[i], 'opts': specific_opts}
def check_plot_observables(index):
    if not index:
        plots_list = []
        for name in plotting_info:
            hv_type = getattr(hv, plotting_info[name]['type'])
            empty_plot = hv_type(np.random.rand(0, 2)).relabel(f'{name} - No Selection').opts(height=400, width=500, fontscale=1.1, framewise=True)
            if plotting_info[name]['opts'] is not None:
                empty_plot.opts(plotting_info[name]['opts'])
            plots_list.append([empty_plot])
    else:
        new_index = [x for x in index if x not in list(plots.keys())]
        for element in new_index:
            new_plots = []
            for each in observables:
                new_plots.extend(each.generate_plot(element))
            plots[element] = {plot.label: plot for plot in new_plots}

        plots_list = []
        for index_item in index:
            plot_types = list(plots[index_item].keys())

        for plot_item in plot_types:
            same_type = [plots[key][plot_item] for key in index]
            plots_list.append(same_type)

    layout = hv.Layout()
    for list_of_plots in plots_list:
        overlay = hv.Overlay(list_of_plots).opts(show_legend=False)
        layout = layout + overlay
    layout.opts(shared_axes=False).cols(3)
    return layout

In [ ]:
check_plot_observables([1,2,3])

In [ ]:
coswav.draw_plot(99)

In [ ]:
xs = np.linspace(0, np.pi*2)
ys = np.sin(xs)
# curve = hv.Curve((xs, ys))
# curve + hv.Histogram(curve)
hv.Histogram((xs, ys))

In [ ]:
hv.help(hv.Bars)